In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import pearsonr
import os
from io import StringIO

# --- AYARLAR ---
sns.set(style="whitegrid")
plt.rcParams['figure.figsize'] = (14, 10)

# --- 1. İSİM DÜZELTME FONKSİYONU ---
def sehir_ismi_duzelt(isim):
    if isinstance(isim, str):
        # TR kodlarını ve boşlukları temizle
        if "TR" in isim:
            if "," in isim: isim = isim.split(',')[1]
            elif " " in isim: isim = isim.split(' ', 1)[1]

        # Türkçe karakterleri düzelt
        isim = isim.replace('i', 'İ').upper()
        isim = isim.replace('ı', 'I')
        return isim.strip()
    return ""

# --- 2. DOSYA OKUMA FONKSİYONU ---
def dosya_oku_koordinatli(anahtar_kelime, satir_baslangic):
    """
    Dosyayı bulur ve belirtilen satırdan itibaren okur.
    satir_baslangic: Excel'deki satır numarası (Python için -1 yapılır).
    """
    dosyalar = os.listdir('.')
    dosya_yolu = next((f for f in dosyalar if anahtar_kelime.lower() in f.lower()), None)

    if not dosya_yolu:
        print(f"⚠️ HATA: '{anahtar_kelime}' dosyası bulunamadı!")
        return None

    print(f"📖 Okunuyor: {dosya_yolu} (Başlangıç Satırı: {satir_baslangic})")

    # Python 0'dan başlar, Excel 1'den. O yüzden skiprows = satir_baslangic - 1
    skip = satir_baslangic - 1

    # Dosya türünü anlamaya çalışarak oku
    try:
        try:
            # CSV (Otomatik)
            df = pd.read_csv(dosya_yolu, skiprows=skip, header=None, sep=None, engine='python')
        except:
            # Excel
            df = pd.read_excel(dosya_yolu, skiprows=skip, header=None)
        return df
    except Exception as e:
        print(f"❌ Dosya okuma hatası: {e}")
        return None

# --- IQR OUTLIER TEMİZLEME FONKSİYONU ---
def iqr_filtrele(df, kolon):
    """Verilen kolona göre aykırı değerleri (Outliers) atar."""
    Q1 = df[kolon].quantile(0.25)
    Q3 = df[kolon].quantile(0.75)
    IQR = Q3 - Q1
    alt_sinir = Q1 - 1.5 * IQR
    ust_sinir = Q3 + 1.5 * IQR

    # Filtrelenmiş veri ve atılan şehir sayısı
    df_clean = df[(df[kolon] >= alt_sinir) & (df[kolon] <= ust_sinir)]
    atilanlar = len(df) - len(df_clean)
    return df_clean, atilanlar

print("🚀 KOORDİNATLI VERİ ANALİZİ BAŞLIYOR...")

try:
    # ---------------------------------------------------------
    # ADIM 1: DOSYALARI KOORDİNATLARA GÖRE OKUMA
    # ---------------------------------------------------------

    # A. HASILA (GSYH)
    # Talimat: Şehir -> 6. Satır B Sütunu (Index 1) | Veri -> 6. Satır AQ Sütunu (Index 42)
    df_gsyh = dosya_oku_koordinatli("hasila", 6)
    if df_gsyh is not None:
        df_gsyh = df_gsyh.iloc[:, [1, 42]]
        df_gsyh.columns = ['Il', 'GSYH_Dolar']
        df_gsyh['Il'] = df_gsyh['Il'].apply(sehir_ismi_duzelt)

    # B. İŞSİZLİK
    # Talimat: Şehir -> 8. Satır B Sütunu (Index 1) | Veri -> 8. Satır H Sütunu (Index 7)
    df_issizlik = dosya_oku_koordinatli("issizlik", 8)
    if df_issizlik is not None:
        df_issizlik = df_issizlik.iloc[:, [1, 7]]
        df_issizlik.columns = ['Il', 'Issizlik_Orani']
        df_issizlik['Il'] = df_issizlik['Il'].apply(sehir_ismi_duzelt)

    # C. SİNEMA
    # Talimat: Şehir -> 6. Satır A Sütunu (Index 0) | Koltuk -> C (Index 2) | İzleyici -> I (Index 8)
    df_sinema = dosya_oku_koordinatli("sinema", 6)
    if df_sinema is not None:
        df_sinema = df_sinema.iloc[:, [0, 2, 8]]
        df_sinema.columns = ['Il', 'Sinema_Koltuk', 'Sinema_Seyirci']
        df_sinema['Il'] = df_sinema['Il'].apply(sehir_ismi_duzelt)

    # D. TİYATRO
    # Talimat: Şehir -> 7. Satır A Sütunu (Index 0) | Koltuk -> C (Index 2) | Seyirci -> M (Index 12)
    df_tiyatro = dosya_oku_koordinatli("tiyatro", 7)
    if df_tiyatro is not None:
        df_tiyatro = df_tiyatro.iloc[:, [0, 2, 12]]
        df_tiyatro.columns = ['Il', 'Tiyatro_Koltuk', 'Tiyatro_Seyirci']
        df_tiyatro['Il'] = df_tiyatro['Il'].apply(sehir_ismi_duzelt)

    # E. NÜFUS (Manuel 2023 Verisi)
    nufus_str = """Il,Nufus
İstanbul,15701602
Ankara,5864049
İzmir,4493242
Bursa,3238618
Antalya,2722103
Konya,2330024
Adana,2280484
Şanlıurfa,2237745
Gaziantep,2193363
Kocaeli,2130006
Mersin,1954279
Diyarbakır,1833684
Hatay,1562185
Manisa,1475353
Kayseri,1452458
Samsun,1382376
Balıkesir,1276096
Tekirdağ,1187162
Aydın,1165943
Kahramanmaraş,1134105
Van,1118087
Sakarya,1110735
Muğla,1081867
Denizli,1061371
Eskişehir,921630
Mardin,895911
Trabzon,822270
Ordu,770711
Malatya,750491
Afyonkarahisar,750193
Erzurum,745005
Batman,654528
Sivas,637007
Tokat,612674
Adıyaman,611037
Elazığ,603941
Zonguldak,586802
Kütahya,571078
Şırnak,570826
Çanakkale,568966
Osmaniye,561061
Çorum,521335
Ağrı,499801
Giresun,455922
Isparta,446409
Aksaray,439474
Edirne,421247
Yozgat,413161
Düzce,412344
Muş,392301
Kastamonu,381991
Kırklareli,379031
Uşak,375310
Niğde,372708
Bitlis,359808
Rize,346977
Amasya,342378
Siirt,336453
Bolu,326409
Nevşehir,317952
Yalova,307882
Bingöl,283276
Kırıkkale,283053
Hakkari,282191
Burdur,275826
Kars,272300
Karaman,262791
Karabük,250478
Kırşehir,244546
Erzincan,241239
Bilecik,228495
Sinop,226957
Iğdır,206857
Bartın,206715
Çankırı,199981
Artvin,169280
Kilis,156739
Gümüşhane,142617
Ardahan,91354
Tunceli,86612
Bayburt,83676"""
    df_nufus = pd.read_csv(StringIO(nufus_str))
    df_nufus['Il'] = df_nufus['Il'].apply(sehir_ismi_duzelt)

    # ---------------------------------------------------------
    # ADIM 2: BİRLEŞTİRME VE HESAPLAMA
    # ---------------------------------------------------------

    # 1. BİRLEŞTİRME
    df_master = df_nufus.merge(df_gsyh, on='Il', how='inner')
    df_master = df_master.merge(df_issizlik, on='Il', how='inner')
    df_master = df_master.merge(df_sinema, on='Il', how='left')
    df_master = df_master.merge(df_tiyatro, on='Il', how='left')

    # 2. TEMİZLİK
    cols = ['GSYH_Dolar', 'Issizlik_Orani', 'Sinema_Koltuk', 'Sinema_Seyirci', 'Tiyatro_Koltuk', 'Tiyatro_Seyirci']
    for col in cols:
        df_master[col] = pd.to_numeric(df_master[col], errors='coerce').fillna(0)

    # "Türkiye" satırlarını çıkar
    df_master = df_master[~df_master['Il'].str.contains("TURKIYE|TOPLAM", na=False)]
    df_master = df_master[df_master['GSYH_Dolar'] > 0]

    # 3. KİŞİ BAŞINA DÜŞEN HESAPLAMALAR
    df_master['KB_Sinema_Koltuk'] = df_master['Sinema_Koltuk'] / df_master['Nufus']
    df_master['KB_Tiyatro_Koltuk'] = df_master['Tiyatro_Koltuk'] / df_master['Nufus']
    df_master['KB_Sinema_Seyirci'] = df_master['Sinema_Seyirci'] / df_master['Nufus']
    df_master['KB_Tiyatro_Seyirci'] = df_master['Tiyatro_Seyirci'] / df_master['Nufus']

    print(f"✅ Analiz için {len(df_master)} il hazırlandı.")

    # ---------------------------------------------------------
    # ADIM 3: İSTATİSTİKSEL ANALİZ VE TABLOLAR
    # ---------------------------------------------------------

    # A. STANDART ANALİZ (Tüm Şehirler)
    results_original = []
    analizler = [
        ('KB_Sinema_Koltuk', 'GSYH_Dolar', 'Sinema Koltuk vs Gelir'),
        ('KB_Sinema_Koltuk', 'Issizlik_Orani', 'Sinema Koltuk vs İşsizlik'),
        ('KB_Tiyatro_Koltuk', 'GSYH_Dolar', 'Tiyatro Koltuk vs Gelir'),
        ('KB_Tiyatro_Koltuk', 'Issizlik_Orani', 'Tiyatro Koltuk vs İşsizlik'),
        ('KB_Sinema_Seyirci', 'GSYH_Dolar', 'Sinema Seyirci vs Gelir'),
        ('KB_Sinema_Seyirci', 'Issizlik_Orani', 'Sinema Seyirci vs İşsizlik'),
        ('KB_Tiyatro_Seyirci', 'GSYH_Dolar', 'Tiyatro Seyirci vs Gelir'),
        ('KB_Tiyatro_Seyirci', 'Issizlik_Orani', 'Tiyatro Seyirci vs İşsizlik'),
    ]

    for y, x, label in analizler:
        r, p = pearsonr(df_master[x], df_master[y])
        results_original.append({'Analiz': label, 'Durum': 'Tüm İller', 'Korelasyon (R)': f"{r:.3f}", 'P-Value': f"{p:.5f}"})

    # B. IQR OUTLIER TEMİZLENMİŞ ANALİZ (Özel İstek)
    results_iqr = []

    # 1. Tiyatro Koltuk vs Gelir
    df_clean_1, atilan_1 = iqr_filtrele(df_master, 'KB_Tiyatro_Koltuk')
    r1, p1 = pearsonr(df_clean_1['GSYH_Dolar'], df_clean_1['KB_Tiyatro_Koltuk'])
    results_iqr.append({'Analiz': 'Tiyatro Koltuk vs Gelir', 'Durum': f'IQR Temiz ({atilan_1} atıldı)', 'Korelasyon (R)': f"{r1:.3f}", 'P-Value': f"{p1:.5f}"})

    # 2. Tiyatro Seyirci vs İşsizlik
    df_clean_2, atilan_2 = iqr_filtrele(df_master, 'KB_Tiyatro_Seyirci')
    r2, p2 = pearsonr(df_clean_2['Issizlik_Orani'], df_clean_2['KB_Tiyatro_Seyirci'])
    results_iqr.append({'Analiz': 'Tiyatro Seyirci vs İşsizlik', 'Durum': f'IQR Temiz ({atilan_2} atıldı)', 'Korelasyon (R)': f"{r2:.3f}", 'P-Value': f"{p2:.5f}"})

    df_res_orig = pd.DataFrame(results_original)
    df_res_iqr = pd.DataFrame(results_iqr)

    # ---------------------------------------------------------
    # ADIM 4: GÖRSELLEŞTİRME
    # ---------------------------------------------------------

    # GRAFİK SETİ 1: STANDART
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    fig.suptitle('GENEL ANALİZ (TÜM İLLER)', fontsize=16)

    sns.regplot(data=df_master, x='GSYH_Dolar', y='KB_Sinema_Seyirci', ax=axes[0,0], color='red')
    axes[0,0].set_title('Sinema Seyirci vs Gelir')

    sns.regplot(data=df_master, x='Issizlik_Orani', y='KB_Sinema_Seyirci', ax=axes[0,1], color='orange')
    axes[0,1].set_title('Sinema Seyirci vs İşsizlik')

    sns.regplot(data=df_master, x='GSYH_Dolar', y='KB_Tiyatro_Seyirci', ax=axes[1,0], color='green')
    axes[1,0].set_title('Tiyatro Seyirci vs Gelir')

    sns.regplot(data=df_master, x='Issizlik_Orani', y='KB_Tiyatro_Seyirci', ax=axes[1,1], color='purple')
    axes[1,1].set_title('Tiyatro Seyirci vs İşsizlik')

    plt.tight_layout()
    plt.show()

    # GRAFİK SETİ 2: IQR TEMİZLENMİŞ
    fig2, axes2 = plt.subplots(1, 2, figsize=(16, 6))
    fig2.suptitle('IQR OUTLIER TEMİZLENMİŞ ANALİZ (ÖZEL İSTEK)', fontsize=16)

    # Tiyatro Koltuk vs Gelir
    sns.regplot(data=df_clean_1, x='GSYH_Dolar', y='KB_Tiyatro_Koltuk', ax=axes2[0], color='teal')
    axes2[0].set_title(f'Tiyatro Koltuk vs Gelir (Temiz)\n{atilan_1} il atıldı')

    # Tiyatro Seyirci vs İşsizlik
    sns.regplot(data=df_clean_2, x='Issizlik_Orani', y='KB_Tiyatro_Seyirci', ax=axes2[1], color='crimson')
    axes2[1].set_title(f'Tiyatro Seyirci vs İşsizlik (Temiz)\n{atilan_2} il atıldı')

    plt.tight_layout()
    plt.show()

    # ---------------------------------------------------------
    # ADIM 5: TABLOLAR
    # ---------------------------------------------------------
    print("\n" + "="*60)
    print("📋 TABLO 1: TÜM İLLER İÇİN GENEL ANALİZ SONUÇLARI")
    print("="*60)
    print(df_res_orig)

    print("\n" + "="*60)
    print("✨ TABLO 2: IQR (OUTLIER TEMİZLENMİŞ) ÖZEL ANALİZ SONUÇLARI")
    print("="*60)
    print(df_res_iqr)
    print("="*60)

except Exception as e:
    print(f"❌ HATA: {e}")